In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\tom\\Google Drive\\University\\Dissertation\\Code'

In [2]:
from gensim.corpora import WikiCorpus, MmCorpus, Dictionary
from gensim import utils

In [3]:
from mywikicorpus import MyWikiCorpus

In [ ]:
from nltk.tokenize import MWETokenizer
from nltk.tokenize import WordPunctTokenizer, WhitespaceTokenizer
import nltk.data

In [ ]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
sent_detector.tokenize("I'm a big banana, you hear? It's all about Mr. Smith.")

In [ ]:
text = "April is the fourth month of the year, and comes between March and May. It is one of four months to have 30 days. \n April always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as ''(December)''. == Events == * First Event * Second Event * '''Linked Event!'''"

sent_detector.tokenize(text)

In [ ]:
import re

reA = re.compile("\'{2,}") # Multiple quotes
reB = re.compile("={2,}")  # Multiple equals
reB2 = re.compile("={2,}$", re.UNICODE)  # Multiple equals, at the end of a token

reC = re.compile("^(\w{0,}[.!?;:\)\(\"])'{2,}$") # Sentence ender followed by double quotes - want to retain only the former

reD = re.compile("\*\s*(\'{0,5}[A-Z])") # Asterisk followed by capital (with optional space) - assume bullet point, sub for \n

def sent_end_rep(matchobj):
    return matchobj.group(1)

def bullet_rep(matchobj):
    return '. '+matchobj.group(1)


def tokenize(content, token_min_len = 1, token_max_len = 15, lower=True):
    #used to override original method in wikicorpus.py
    outlist = []
    # Sentences
    for sent in sent_detector.tokenize(re.sub(reD,bullet_rep,content)): # Asterisk followed by capital - assume bullet marker, swap * for .
        for t in WhitespaceTokenizer().tokenize(sent):
            if len(t) >= token_min_len and len(t) <= token_max_len and not t.startswith('_'):
                if re.fullmatch(reA,t):  # Omit markup tokens of multiple quotes
                    continue
                elif re.fullmatch(reB,t): # Multiple equals (heading markup) -> newline
                    t = '\n'
                elif re.fullmatch(reC,t):
                    t = re.sub(reC,sent_end_rep,t) # Retain sentence enders followed by double quotes
                    
                t = re.sub(reA,'',t) # Remove blocks of multiple quotes (from markup)
                t = re.sub(reB2,'\n',t) # Multiple equals at end of a token should be a newline (to keep headings as sentences)
                t = re.sub(reB,'',t) # Remove blocks of multiple equals (from markup)
                
                if lower:
                    outlist.append(utils.to_unicode(t.lower()))
                else:
                    outlist.append(utils.to_unicode(t))
                    
        outlist.append('\n')
    return outlist

In [ ]:
reC.match("Bums!'''").group(1)

In [ ]:
def sent_end_rep(matchobj):
    return matchobj.group(1)
    
re.sub(reC,sent_end_rep,"!'''")

In [ ]:
tokenize(text, lower=False)

In [ ]:
tokenize("self-conscious bums")

In [4]:
simp = MyWikiCorpus(path+'/Data/Corpora/wiki/simplewiki-20200601-pages-articles-multistream.xml.bz2',
                  lower=False,
                  token_min_len = 1,
                  token_max_len = 40
                 )


C:\Anaconda\envs\py_36\lib\site-packages\gensim\utils.py:1254: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
i = 0

for text in simp.get_texts():
    print(text)
    print('\n\n')
    i += 1
    if i >= 5: break

['April', 'is', 'the', 'fourth', 'month', 'of', 'the', 'year,', 'and', 'comes', 'between', 'March', 'and', 'May.', '\n', 'It', 'is', 'one', 'of', 'four', 'months', 'to', 'have', '30', 'days.', '\n', 'April', 'always', 'begins', 'on', 'the', 'same', 'day', 'of', 'week', 'as', 'July,', 'and', 'additionally,', 'January', 'in', 'leap', 'years.', '\n', 'April', 'always', 'ends', 'on', 'the', 'same', 'day', 'of', 'the', 'week', 'as', 'December.', '\n', "April's", 'flowers', 'are', 'the', 'Sweet', 'Pea', 'and', 'Daisy.', '\n', 'Its', 'birthstone', 'is', 'the', 'diamond.', '\n', 'The', 'meaning', 'of', 'the', 'diamond', 'is', 'innocence.', '\n', '\n', 'The', 'Month', '\n', 'Spring', 'flowers', 'in', 'April', 'in', 'the', 'Northern', 'Hemisphere.', '\n', 'April', 'comes', 'between', 'March', 'and', 'May,', 'making', 'it', 'the', 'fourth', 'month', 'of', 'the', 'year.', '\n', 'It', 'also', 'comes', 'first', 'in', 'the', 'year', 'out', 'of', 'the', 'four', 'months', 'that', 'have', '30', 'days,',

In [6]:
output = open(path+'/Data/Corpora/wiki/simple_20200601/simple_20200601_v2.txt', 'w', encoding='utf-8')

i = 0
for text in simp.get_texts():
    output.write(bytes(' '.join(text), 'utf-8').decode('utf-8')+'\n')
    i = i + 1
    if (i % 10000 == 0):
        print('Processed ' + str(i) + ' articles')
        
output.close()
print('Processing complete!')

Processed 10000 articles
Processed 20000 articles
Processed 30000 articles
Processed 40000 articles
Processed 50000 articles
Processed 60000 articles
Processed 70000 articles
Processed 80000 articles
Processed 90000 articles
Processed 100000 articles
Processed 110000 articles
Processed 120000 articles
Processing complete!
